In [6]:
# https://github.com/prediqtiv/alpha-vantage-cookbook/blob/master/data/NASDAQ.txt

In [5]:
import pandas as pd
import time
import requests
from google.cloud import bigquery

In [6]:
client = bigquery.Client()
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True

In [7]:
class KeyExpiredException(Exception):
    pass
    
def getFinancials(symbol, function):
    """
    service = [BALANCE_SHEET, DIVIDENDS, INCOME_STATEMENT, CASH_FLOW, EARNINGS]
    """
    API_url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={API_key}'
    data = requests.get(API_url).json()    
    if data == {}:
        raise Exception("No data avalaible")

    if "Error Message" in data:
        raise Exception(data["Error Message"])
        
        
    if keyExperied(data):
        raise KeyExpiredException(data["Information"])

    
    if function == "DIVIDENDS":
        df = pd.DataFrame(data["data"])
        df["symbol"] = data["symbol"]
        return df

    if function == "EARNINGS":
        df_year = pd.DataFrame(data["annualEarnings"])
        df_year["reporting"] = "Y"
        df_quat = pd.DataFrame(data["quarterlyEarnings"])
        df_quat["reporting"] = "Q"
        df = pd.concat([df_year, df_quat], ignore_index=True)
        df["symbol"] = data["symbol"]
        return df
    
    
    df_year = pd.DataFrame(data["annualReports"])
    df_year["reporting"] = "Y"
    df_quat = pd.DataFrame(data["quarterlyReports"])
    df_quat["reporting"] = "Q"
    df = pd.concat([df_year, df_quat], ignore_index=True)
    df["symbol"] = data["symbol"]
    return df


def keyExperied(res):
    return "Information" in res and "rate limit" in res["Information"]  


def persist(df, financial):
    datasetName = "financials"
    dataset  = client.dataset(datasetName)
    table = dataset.table(financial)
    job = client.load_table_from_dataframe(df, table, job_config=job_config)
    job.result()

def isProcessed(symbol, function):
    query_job = client.query(f'select symbol from `fiancee.financials.{function}` where symbol = "{symbol}"')  # API request
    if len(list(query_job.result())) > 0:
        return True
    query_job = client.query(f'select symbol from `fiancee.financials.metadata` where symbol = "{symbol}" and report = "{function}"')  # API request
    if len(list(query_job.result())) > 0:
        return True
    return False

In [9]:
url = "https://raw.githubusercontent.com/prediqtiv/alpha-vantage-cookbook/master/data/NASDAQ.txt"
df = pd.read_csv(url, delimiter='\t')
_df = df.head(1000)

# symbols = _df.Symbol.values
symbols = ["CCJ", "ZAL", "UEC"]

for symbol in symbols:
    print(symbol)
    isExpired = False
    for fun in ["BALANCE_SHEET", "DIVIDENDS", "INCOME_STATEMENT", "CASH_FLOW", "EARNINGS"]:
        if not isProcessed(symbol, fun):
            try:
                df = getFinancials(symbol, fun)
                persist(df, fun) 
            except KeyExpiredException as e:
                print("Key expired:", API_key)
                isExpired = True
                break
            except Exception as e:
                persist(pd.DataFrame([{"symbol" : symbol, "report" : fun, "avalaible" : False}]) , "metadata") 
                print("Failed to process:", symbol, fun, "due to error", e)
        else:
            print(symbol, fun, " is already processed")

    if isExpired:
        print("Finish processing")
        break
        
        


CCJ
ZAL
Failed to process: ZAL BALANCE_SHEET due to error No data avalaible
Failed to process: ZAL INCOME_STATEMENT due to error No data avalaible
Failed to process: ZAL CASH_FLOW due to error No data avalaible
Failed to process: ZAL EARNINGS due to error No data avalaible
UEC


In [11]:
# function = "BALANCE_SHEET"
# symbol = "AABA"
# API_key = "F5UOTYQ2089J55ZA"
# API_url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={API_key}'
# data = requests.get(API_url).json()    


In [12]:
# data
# pd.DataFrame(data["annualReports"])

In [13]:
# res = {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
# "rate limit" in res["Information"] 

In [14]:
# pd.DataFrame([{"symbol" : "AAAP", "report" : "BALAB", "avalaible" : True}]) 


In [15]:
persist(pd.DataFrame([{"symbol" : "TEST", "report" : "BALANCE_SHEET", "avalaible" : False}]) , "metadata") 